In [74]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 
from pathlib import Path


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

paths = Path(args.train_folder).glob('**/chorale_*.csv')
train_np_list = [pd.read_csv(p).values.tolist() for p in paths]
# print(len(train_np_list[0]))
# print(train_np_list[0])

paths = Path(args.val_folder).glob('**/chorale_*.csv')
val_np_list = [pd.read_csv(p).values.tolist() for p in paths]

paths = Path(args.test_folder).glob('**/chorale_*.csv')
test_np_list = [pd.read_csv(p).values.tolist() for p in paths]

def calc_const():
    notes = set()
    for chorales in (train_np_list, val_np_list, test_np_list):
        for chorale in chorales:
            for chord in chorale:
                notes |= set(chord)

    n_notes = len(notes)
    min_note = min(notes - {0})
    max_note = max(notes)
    return n_notes, min_note, max_note

n_notes, min_note, max_note = calc_const()


def create_target(batch):
    print(batch)
    X = batch[:, :-1]
    Y = batch[:, 1:] # predict next note in each arpegio, at each step
    return X, Y

def preprocess(window):
    window = tf.where(window == 0, window, window - min_note + 1) # shift values
    return tf.reshape(window, [-1]) # convert to arpegio

def bach_dataset(chorales, batch_size=32, shuffle_buffer_size=None,
                 window_size=32, window_shift=16, cache=True):
    def batch_window(window):
        return window.batch(window_size + 1)

    def to_windows(chorale):
        dataset = tf.data.Dataset.from_tensor_slices(chorale)
        dataset = dataset.window(window_size + 1, window_shift, drop_remainder=True)
        dataset = dataset.flat_map(batch_window)
        return dataset

    chorales = tf.ragged.constant(chorales, ragged_rank=1)
#     print("chorale[0] {}".format(chorales[0]))
    dataset = tf.data.Dataset.from_tensor_slices(chorales)
    dataset = dataset.flat_map(to_windows)
    for d in dataset:
        tf.print(f"d after flat_map: {d}")
        break
    dataset = dataset.map(preprocess)
    for d in dataset:
        tf.print(f"d after preprocess: {d}")
        break
    
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(create_target)
    for d in dataset:
        print(f"final dataset: {d}")
        break
    return dataset.prefetch(1)


train_chorales = bach_dataset(np_list)
val_chorales = bach_dataset(np_list)
test_chorales = bach_dataset(np_list)

d after flat_map: [[66 61 57 54]
 [66 61 57 54]
 [68 61 59 54]
 [68 61 59 54]
 [69 66 61 54]
 [69 66 61 56]
 [69 66 61 57]
 [69 66 61 59]
 [68 65 61 61]
 [68 65 61 61]
 [68 65 59 49]
 [68 65 59 49]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 59 50]
 [66 66 59 50]
 [68 66 59 50]
 [68 66 59 50]
 [69 66 61 49]
 [69 66 61 49]
 [69 66 61 47]
 [69 66 61 47]
 [68 65 61 49]
 [68 65 61 49]
 [68 65 59 49]
 [68 65 59 49]
 [66 61 57 42]
 [66 61 57 42]
 [66 61 57 42]
 [66 61 57 42]
 [73 66 57 54]]
d after preprocess: [31 26 22 19 31 26 22 19 33 26 24 19 33 26 24 19 34 31 26 19 34 31 26 21
 34 31 26 22 34 31 26 24 33 30 26 26 33 30 26 26 33 30 24 14 33 30 24 14
 31 31 22 15 31 31 22 15 31 31 22 15 31 31 22 15 31 31 24 15 31 31 24 15
 33 31 24 15 33 31 24 15 34 31 26 14 34 31 26 14 34 31 26 12 34 31 26 12
 33 30 26 14 33 30 26 14 33 30 24 14 33 30 24 14 31 26 22  7 31 26 22  7
 31 26 22  7 31 26 22  7 38 31 22 19]
Tensor("args_0:0", shape=(None, None), dtype=int32)
final datas

2021-09-14 21:49:40.654408: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


d after flat_map: [[66 61 57 54]
 [66 61 57 54]
 [68 61 59 54]
 [68 61 59 54]
 [69 66 61 54]
 [69 66 61 56]
 [69 66 61 57]
 [69 66 61 59]
 [68 65 61 61]
 [68 65 61 61]
 [68 65 59 49]
 [68 65 59 49]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 59 50]
 [66 66 59 50]
 [68 66 59 50]
 [68 66 59 50]
 [69 66 61 49]
 [69 66 61 49]
 [69 66 61 47]
 [69 66 61 47]
 [68 65 61 49]
 [68 65 61 49]
 [68 65 59 49]
 [68 65 59 49]
 [66 61 57 42]
 [66 61 57 42]
 [66 61 57 42]
 [66 61 57 42]
 [73 66 57 54]]
d after preprocess: [31 26 22 19 31 26 22 19 33 26 24 19 33 26 24 19 34 31 26 19 34 31 26 21
 34 31 26 22 34 31 26 24 33 30 26 26 33 30 26 26 33 30 24 14 33 30 24 14
 31 31 22 15 31 31 22 15 31 31 22 15 31 31 22 15 31 31 24 15 31 31 24 15
 33 31 24 15 33 31 24 15 34 31 26 14 34 31 26 14 34 31 26 12 34 31 26 12
 33 30 26 14 33 30 26 14 33 30 24 14 33 30 24 14 31 26 22  7 31 26 22  7
 31 26 22  7 31 26 22  7 38 31 22 19]
Tensor("args_0:0", shape=(None, None), dtype=int32)
final datas

2021-09-14 21:49:41.837426: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


d after flat_map: [[66 61 57 54]
 [66 61 57 54]
 [68 61 59 54]
 [68 61 59 54]
 [69 66 61 54]
 [69 66 61 56]
 [69 66 61 57]
 [69 66 61 59]
 [68 65 61 61]
 [68 65 61 61]
 [68 65 59 49]
 [68 65 59 49]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 59 50]
 [66 66 59 50]
 [68 66 59 50]
 [68 66 59 50]
 [69 66 61 49]
 [69 66 61 49]
 [69 66 61 47]
 [69 66 61 47]
 [68 65 61 49]
 [68 65 61 49]
 [68 65 59 49]
 [68 65 59 49]
 [66 61 57 42]
 [66 61 57 42]
 [66 61 57 42]
 [66 61 57 42]
 [73 66 57 54]]
d after preprocess: [31 26 22 19 31 26 22 19 33 26 24 19 33 26 24 19 34 31 26 19 34 31 26 21
 34 31 26 22 34 31 26 24 33 30 26 26 33 30 26 26 33 30 24 14 33 30 24 14
 31 31 22 15 31 31 22 15 31 31 22 15 31 31 22 15 31 31 24 15 31 31 24 15
 33 31 24 15 33 31 24 15 34 31 26 14 34 31 26 14 34 31 26 12 34 31 26 12
 33 30 26 14 33 30 26 14 33 30 24 14 33 30 24 14 31 26 22  7 31 26 22  7
 31 26 22  7 31 26 22  7 38 31 22 19]
Tensor("args_0:0", shape=(None, None), dtype=int32)
final datas

2021-09-14 21:49:43.022477: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
def build_model():
    n_embedding_dims = 5

    model = keras.models.Sequential([
        keras.layers.Embedding(input_dim=n_notes, output_dim=n_embedding_dims,
                               input_shape=[None]),
        keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
        keras.layers.BatchNormalization(),
        keras.layers.LSTM(256, return_sequences=True),
        keras.layers.Dense(n_notes, activation="softmax")
    ])

    log.info(model.summary())
    return model

def train_model(model):  
    optimizer = keras.optimizers.Nadam(learning_rate=1e-3)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    model.fit(train_chorales, epochs=20, validation_data=val_chorales)
model = build_model()
train_model(model)
model.save("model-ignored//my_bach_model.h5")
model.evaluate(test_chorales)

2021-09-14 21:53:54,351 : INFO : None


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 5)           235       
_________________________________________________________________
conv1d_12 (Conv1D)           (None, None, 32)          352       
_________________________________________________________________
batch_normalization_12 (Batc (None, None, 32)          128       
_________________________________________________________________
conv1d_13 (Conv1D)           (None, None, 48)          3120      
_________________________________________________________________
batch_normalization_13 (Batc (None, None, 48)          192       
_________________________________________________________________
conv1d_14 (Conv1D)           (None, None, 64)          6208      
_________________________________________________________________
batch_normalization_14 (Batc (None, None, 64)         

### Tests
#### Dataset creation

In [53]:
tf.ragged.constant([[1, 2], [3], [4, 5, 6]], ragged_rank=1)
tf.ragged.constant([[[1, 2], [3, 2], [4, 5]], [[1, 2]]], ragged_rank=2)
# tf.ragged.constant([[[1], [2]], [[3], [4]], [[5], [6]]], ragged_rank=1)

<tf.RaggedTensor [[[1, 2], [3, 2], [4, 5]], [[1, 2]]]>

In [15]:
dataset = tf.data.Dataset.range(7).window(3, 2)
for d in dataset:
    print(d)
    print([item.numpy() for item in d])

<_VariantDataset shapes: (), types: tf.int64>
[0, 1, 2]
<_VariantDataset shapes: (), types: tf.int64>
[2, 3, 4]
<_VariantDataset shapes: (), types: tf.int64>
[4, 5, 6]
<_VariantDataset shapes: (), types: tf.int64>
[6]


In [55]:
dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4], [5, 6], [7, 8]]).window(2, 1, drop_remainder=True)
for d in dataset:
    print(d)
    print([item.numpy() for item in d])
    
def b(window):
    return window.batch(3)

dataset = dataset.flat_map(b)
print("flat_map")
for d in dataset:
    print(d)
    print([item.numpy() for item in d])

def b1(window):
    window = tf.where(window == 1, window, window - 1) # shift values
    tf.print(window)
    return window

print("map1")
dataset = dataset.map(b1)
for d in dataset:
    print(d)
    print([item.numpy() for item in d])
    
def b2(window):
    window = tf.reshape(window, [-1])
    tf.print(window)
    return window

print("map2")
dataset = dataset.map(b2)
for d in dataset:
    print(d)
    print([item.numpy() for item in d])

<_VariantDataset shapes: (2,), types: tf.int32>
[array([1, 2], dtype=int32), array([3, 4], dtype=int32)]
<_VariantDataset shapes: (2,), types: tf.int32>
[array([3, 4], dtype=int32), array([5, 6], dtype=int32)]
<_VariantDataset shapes: (2,), types: tf.int32>
[array([5, 6], dtype=int32), array([7, 8], dtype=int32)]
flat_map
tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
[array([1, 2], dtype=int32), array([3, 4], dtype=int32)]
tf.Tensor(
[[3 4]
 [5 6]], shape=(2, 2), dtype=int32)
[array([3, 4], dtype=int32), array([5, 6], dtype=int32)]
tf.Tensor(
[[5 6]
 [7 8]], shape=(2, 2), dtype=int32)
[array([5, 6], dtype=int32), array([7, 8], dtype=int32)]
map1
[[1 1]
 [2 3]]
tf.Tensor(
[[1 1]
 [2 3]], shape=(2, 2), dtype=int32)
[array([1, 1], dtype=int32), array([2, 3], dtype=int32)]
[[2 3]
 [4 5]]
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)
[array([2, 3], dtype=int32), array([4, 5], dtype=int32)]
[[4 5]
 [6 7]]
tf.Tensor(
[[4 5]
 [6 7]], shape=(2, 2), dtype=int32)
[array([4, 5], dt